In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from __future__ import division
import helper_functions
import textract
import string, re

# For Polishing Data Cleaning Pipeline
Test before implementing in `helper_function.py`.
<br><br>

In [ ]:
filename = '../raw-data/suspectA/suspectA-1.pdf'

In [ ]:
doc = textract.process(filename, method='pdftotext', layout=True,
                           encoding='ascii')                            # output encoding; input encoding is inferred


In [ ]:

# ---simple data cleaning to remove artifacts/equations/text from figures/flowcharts---

# line break characters (string.whitespace = '\t', '\n', '\x0b', '\x0c', '\r', ' ')
# \x0c = a form feed; it forces a printer to move to the next sheet of paper.
breaks = tuple(ch for ch in tuple(string.whitespace)[1:-1] if doc.count(ch) > 0)

# remove artifact from justified text alignment and change four spaces to '\t' (tab)
doc = re.sub('    (?=\w)', '\t', doc.replace('\n         ', '\n'))

# drop references if exist
i = doc.find('References') * doc.find('Bibliography')  # each returns -1 if not found
if i < 0:
    doc = doc[:-i]


In [ ]:
doc

In [ ]:
# split on line break characters
sentences = re.split('|'.join(breaks), doc)

In [ ]:
sentences

In [ ]:
sentences2 = [s.strip(' ') for s in sentences]

In [ ]:
sentences2

In [ ]:
# remove leading/trailing ' ' (note: can't leave strip() empty otherwise \t is stripped as well!)
# remove page number
# drop empty sentences and sentences with too many white characters in it (most likely text from flowcharts)
# drop lines where there are more digits than alphabets -- most likely an equation
table = string.maketrans('', '')
sentences3 = [s.strip(' ') for s in sentences if s and \
                                     not s.isdigit() and \
                                     0 < s.count(' ')/len(s) < 0.2 and \
                                     len(s.translate(table, string.digits + string.punctuation))/len(s) > 0.5]

In [ ]:
sentences3

In [ ]:
sentences4 = helper_functions.strip_misc_periods(sentences3)

In [ ]:
sentences4

In [ ]:
doc2 = ' '.join(sentences4) 
doc2 = re.sub('\'ll', ' will', doc2)
doc2 = re.sub('\'ve', ' have' , doc2)
doc2 = re.sub('n\'t', ' not' , doc2)
doc2 = re.sub('\w+[a-z]([A-Z]|\_\w)\w+', '', doc2)
# doc2 = re.sub(r'[^\x00-\x7F]+',' ', doc2)

In [ ]:
import numpy as np

In [ ]:
words = np.array([word for word in \
                 re.split(r'[^\x00-\x7F]|[' + string.whitespace + string.punctuation + string.digits + ']', doc2.lower()) \
                 if word \
                    and (len(word) > 1 or word in {'a', 'i', 'A', 'I'}) \
                    and word not in {'satoshi', 'nakamoto',
                                     'suspectA-firstname', 'suspectA-lastname',
                                     'suspectB-firstname', 'suspectB-lastname',
                                     'suspectC-firstname', 'suspectC-lastname'}])

In [ ]:
words[:100]

In [ ]:
import enchant
d = enchant.Dict('en_US')
d2 = enchant.Dict('en_GB')

In [ ]:
isUS = np.array([d.check(word) or d.check(word.capitalize()) or d.check(word.upper()) for word in words])

In [ ]:
isGB = np.array([d2.check(word) or d2.check(word.capitalize()) or d2.check(word.upper()) for word in words])

In [ ]:
not_en = words[~isUS&~isGB][:100]
words = np.append(words, [t1+t2 for t1,t2 in zip(not_en[:-1], not_en[1:]) if d.check(t1+t2)])

In [ ]:
import spacy
from spacy.en import English
tagger = English()

In [ ]:
tagger(u' '.join(words[isUS|isGB]))

In [ ]:
d.check('Katz')

In [ ]:
tokens = [w for w in tagger(u' '.join(words[isUS|isGB])) if w.pos_ not in {u'X', u'PUNCT'}]

In [ ]:
print [(w.orth_, w.pos_) for w in tagger(u' '.join(words[isUS|isGB][:200]))]

In [ ]:
[w.lemma_ for w in tokens]

In [ ]:
re.sub('\w+[a-z]([A-Z]|\_\w)\w+', '', 'getProperties get_propertiesNow_now but this is ok')


In [ ]:
re.sub('\w+[a-z]([A-Z]|\_\w)*', '', 'getProperties get_PropertiesNow_now but this is ok')

In [ ]:
string.printable

In [ ]:
string.letters